In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
#import chromedriver_binary
import lxml.html
import pandas as pd

import re
import datetime
from time import sleep
from argparse import ArgumentParser


In [ ]:
# クローリング・スクレイピングのメイン関数
#def get_win_odds_list(open_date, output_mode, target_race_course, target_race_no):
def get_win_odds_list(open_info, output_mode='D'):
    # pandasの標準出力の桁揃えオプション
    pd.set_option('display.unicode.east_asian_width', True)

    # 引数事前処理
    #open_info               = get_open_info(open_date)                  # 対象日付より開催情報を取得
    #target_race_course_code = conversion_racecourse(target_race_course) # 競馬場名をコードに変換
    #target_race_no_fill     = str(target_race_no).zfill(2)              # レースNoを2桁に変換

    # メイン処理
    win_odds_df = pd.DataFrame()
    for race_place in open_info:
        win_odds_url = 'https://race.netkeiba.com/odds/index.html?race_id=' + race_place
        print(win_odds_url)
        sleep(5)
        # 出力モードSの場合は標準出力にprint、出力モードDの場合はDataFrame形式でreturn
        if output_mode == 'S':
            print(get_win_odds_df(win_odds_url))
        elif output_mode == 'D':
            win_odds_dict = get_win_odds_df(win_odds_url)
            win_odds_dict['netkeiba_race_id'] = race_place
            new_df = pd.DataFrame(win_odds_dict, columns=['horse_number', 'netkeiba_race_id', '馬名', '今回単勝オッズ'])
            win_odds_df = pd.concat([win_odds_df, new_df])
        else:
            print(f'無効なパラメータ {output_mode}')
            
    return win_odds_df

# 単勝オッズページを引数に指定して、そのページから馬番・馬名・単勝オッズを取得し、DataFrame形式で返す
def get_win_odds_df(url):
    # Seleniumを用いてクローリング
    binary = FirefoxBinary('/usr/bin/firefox')
    binary.add_command_line_options('-headless')
    driver = webdriver.Firefox(firefox_binary=binary)
    #options = webdriver.ChromeOptions()
    #options.add_argument('--headless')
    #driver = webdriver.Chrome(options=options)
    #driver = webdriver.Chrome()                  # PhamtomJSのWebDriverオブジェクトを作成する。
    #driver = webdriver.PhantomJS(executable_path='/usr/local/bin/phantomjs', service_log_path='/tmp/ghostdriver.log')
    driver.get(url)                                 # オッズ表示画面を開く
    sleep(5)                                        # 負荷分散の為のsleep
    root = lxml.html.fromstring(driver.page_source) # 検索結果を表示し、lxmlで解析準備
    

    ##htmltag = odds.xpath("div[@class='OneTable']")
    #dir(odds)
    horses = root.cssselect('.Horse_Name')[0].xpath("//span[contains(@id, 'ninki-name')]")
    tansyos = root.cssselect('.Odds')[0].xpath("//span[contains(@id, 'odds')]")
    horses_size = len(horses)
    tansyos_size = len(tansyos)

    if horses_size != tansyos_size:
        print(f'データに不備があるかもしれません。horses size={horses_size}, tansyos size={tansyos_size}')

    # 辞書・リストの初期化
    win_odds_dict     = {}
    horse_number_list = []
    horse_name_list   = []
    tan_odds_list     = []
    #win_odds_df       = pd.DataFrame()

    for i in range(horses_size):
        try:
            #print(i+1, horses[i].text, tansyos[i].text)
            horse_number_list.append(str(i+1))
            horse_name_list.append(horses[i].text)
            tan_odds_list.append(tansyos[i].text)
        #dl = odds.xpath('//html/body/div[1]/div[2]/div[1]')
        #print('dl:', len(dl))
        except IndexError as e:
            print('IndexError: ', e, name)
            break
        except:
            print('Unkown Error: ', name)
            continue
        finally: # defer
            pass

    win_odds_dict["horse_number"] = horse_number_list
    win_odds_dict["馬名"]   = horse_name_list
    win_odds_dict["今回単勝オッズ"]     = tan_odds_list
    #win_odds_df                   = pd.DataFrame(win_odds_dict,columns=['horse_number', 'horse_name', 'tan_odds'])

    driver.quit()
    return win_odds_dict


# 単勝オッズ取得対象ページのURLを整形する
def format_win_odds_url(race_course,race_times,race_day,race_no):
    
    win_odds_url_template = "http://race.netkeiba.com/?pid=odds&id=p"
    year = datetime.date.today().year

    win_odds_url = win_odds_url_template + str(year) + str(race_course) + str(race_times) + str(race_day) + str(race_no)

    return win_odds_url

In [ ]:
# netkeiba.comより予想オッズを取得するスクレイパー
#
# クローリング・スクレイピング実行
# 引数1： netkeibaのレースID　List型
# 引数2： S = アウトプットとして出力する
#        D = DataFrameとして返却する
#
#get_win_odds_list(open_date, output_mode, target_race_course, target_race_no)
open_race = ['202209020611']
odds = get_win_odds_list(open_race)
odds

In [ ]:
odds.to_csv(f'{open_race[0]}_main.csv', index=False)